### Install libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.33.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00


### Imports and Setup

In [ ]:
import os
import torch
import transformers
import pandas as pd
import torch.nn as nn
import bitsandbytes as bnb
import matplotlib.pyplot as plt

from trl import SFTTrainer
from huggingface_hub import login
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging

### Parameters

In [ ]:
# Model and dataset information
DEVICE_MAP = {"": 0}
DATASET = "Hawk28/spider-sql-prompts"
FINETUNED_MODEL = "Hawk28/CodeLlama-13B-RP"
BASE_MODEL_NAME = "codellama/CodeLlama-13b-hf"

# QLoRA parameters
LORA_R = 8
LORA_ALPHA = 16
LORA_BIAS = "none"
LORA_DROPOUT = 0.05
LORA_TASK_TYPE = "CAUSAL_LM"
LORA_TARGET_MODULES = ['q_proj','k_proj','v_proj','o_proj']

# Bitsandbytes parameters"
BNB_USE_4_BIT = True
BNB_USE_NESTED_QUANT = False
BNB_4_BIT_QUANT_TYPE = "nf4"
BNB_4_BIT_COMPUTE_DTYPE = "float16"

# Trainer parameters
TRAINER_FP16 = False
TRAINER_BF16 = True
TRAINER_MAX_STEPS = 300
TRAINER_EVAL_STEPS = 25
TRAINER_LOGGING_STEPS = 25
TRAINER_MAX_GRAD_NORM = 0.3
TRAINER_WARMUP_RATIO = 0.05
TRAINER_WEIGHT_DECAY = 0.001
TRAINER_NUM_TRAIN_EPOCHS = 1
TRAINER_LEARNING_RATE = 2e-4
TRAINER_GROUP_BY_LENGTH = True
TRAINER_OUTPUT_DIR = "./results"
TRAINER_OPTIM = "paged_adamw_32bit"
TRAINER_LR_SCHEDULER_TYPE = "linear"
TRAINER_EVALUATION_STRATEGY = "steps"
TRAINER_GRADIENT_CHECKPOINTING = True
TRAINER_PER_DEVICE_TRAIN_BATCH_SIZE = 4
TRAINER_GRADIENT_ACCUMULATION_STEPS = 2

# SFT Parameters
SFT_PACKING = False
SFT_MAX_SEQ_LENGTH = 1024

### Data

In [ ]:
dataset = load_dataset(DATASET)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['db_name', 'prompt'],
        num_rows: 8659
    })
    validation: Dataset({
        features: ['db_name', 'prompt'],
        num_rows: 1034
    })
})

In [ ]:
print(dataset["train"][100]["prompt"])

### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Addresses ( address_id, line_1, line_2, city, zip_postcode, state_province_county, country )
# People ( person_id, first_name, middle_name, last_name, cell_mobile_number, email_address, login_name, password )
# Students ( student_id, student_details )
# Courses ( course_id, course_name, course_description, other_details )
# People_Addresses ( person_address_id, person_id, address_id, date_from, date_to )
# Student_Course_Registrations ( student_id, course_id, registration_date )
# Student_Course_Attendance ( student_id, course_id, date_of_attendance )
# Candidates ( candidate_id, candidate_details )
# Candidate_Assessments ( candidate_id, qualification, assessment_date, asessment_outcome_code )
# 
# Students.student_id can be joined with People.person_id
# People_Addresses.address_id can be joined with Addresses.address_id
# People_Addresses.person_id can be joined with People.pe

### Bits and Bytes Configuration

In [ ]:
BNB_4_BIT_COMPUTE_DTYPE = getattr(torch, BNB_4_BIT_COMPUTE_DTYPE)
BNB_4_BIT_COMPUTE_DTYPE

torch.float16

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = BNB_USE_4_BIT,
    bnb_4bit_quant_type = BNB_4_BIT_QUANT_TYPE,
    bnb_4bit_compute_dtype = BNB_4_BIT_COMPUTE_DTYPE,
    bnb_4bit_use_double_quant = BNB_USE_NESTED_QUANT
)

In [ ]:
# Check GPU compatibility with bfloat16
if BNB_4_BIT_COMPUTE_DTYPE == torch.float16 and BNB_USE_4_BIT:
  major, _ = torch.cuda.get_device_capability()
  if major >= 8:
    print("=" * 80)
    print("Your GPU supports bfloat16: accelerate training with bf16=True")
    print("=" * 80)
    TRAINER_BF16 = True

Your GPU supports bfloat16: accelerate training with bf16=True


### Load Base Model and Tokenizer

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    device_map = DEVICE_MAP,
    quantization_config = bnb_config,
    trust_remote_code = True
)
model.config.use_cache = False
model.config.pretraining_tp = 1

(…)odeLlama-13b-hf/resolve/main/config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/31.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)b-hf/resolve/main/generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

(…)3b-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)Llama-13b-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# continents ( ContId, Continent )
# countries ( CountryId, CountryName, Continent )
# car_makers ( Id, Maker, FullName, Country )
# model_list ( ModelId, Maker, Model )
# car_names ( MakeId, Model, Make )
# cars_data ( Id, MPG, Cylinders, Edispl, Horsepower, Weight, Accelerate, Year )
#
# countries.Continent can be joined with continents.ContId
# car_makers.Country can be joined with countries.CountryId
# model_list.Maker can be joined with car_makers.Id
# car_names.Model can be joined with model_list.Model
# cars_data.Id can be joined with car_names.MakeId
#
### Question:
#
# For each continent, list its id, name, and how many countries it has?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# continents ( ContId, Continent )
# countries ( CountryId, CountryName, Continent )
# car_makers ( Id, Maker, FullName, Country )
# model_list ( ModelId, Maker, Model )
# car_names ( MakeId, Model, Make )
# cars_data ( Id, MPG, Cylinders, Edispl, Horsepower, Weight, Accelerate, Year )
#
# countries.Continent can be joined with continents.ContId
# car_makers.Country can be joined with countries.CountryId
# model_list.Maker can be joined with car_makers.Id
# car_names.Model can be joined with model_list.Model
# cars_data.Id can be joined with car_names.MakeId
#
### Question:
#
# For each continent, list its id, name, and how many countries it has?
#
### SQL:
#
# SELECT continents.ContId, continents.Continent, COUNT(countries.CountryId)
# FROM continents
# LEFT JOIN countries ON continents.ContId = countries.Continent
# GROUP BY continents.ContId, continents.Continent
#
### Result

In [ ]:
# batch = tokenizer("""Below is an instruction that describes a task, paired with an input that provides further context. Write an SQL query that appropriately completes the request. Make sure to use the table relationships when joining two or more tables.

# ### Instruction:

# For each continent, list its id, name, and how many countries it has?

# ### Input:

# CREATE TABLE continents (ContId NUMBER PRIMARY KEY, Continent TEXT); CREATE TABLE countries (CountryId NUMBER PRIMARY KEY, CountryName TEXT, Continent NUMBER); CREATE TABLE car_makers (Id NUMBER PRIMARY KEY, Maker TEXT, FullName TEXT, Country TEXT); CREATE TABLE model_list (ModelId NUMBER PRIMARY KEY, Maker NUMBER, Model TEXT); CREATE TABLE car_names (MakeId NUMBER PRIMARY KEY, Model TEXT, Make TEXT); CREATE TABLE cars_data (Id NUMBER PRIMARY KEY, MPG TEXT, Cylinders NUMBER, Edispl NUMBER, Horsepower TEXT, Weight NUMBER, Accelerate NUMBER, Year NUMBER);

# ### Table Relationships:

# countries Continent RELATES TO continents ContId; car_makers Country RELATES TO countries CountryId; model_list Maker RELATES TO car_makers Id; car_names Model RELATES TO model_list Model; cars_data Id RELATES TO car_names MakeId

# ### SQL:
# """, return_tensors='pt')

# with torch.cuda.amp.autocast():
#     input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
#     output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

# print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

### LoRA Configuration

In [ ]:
peft_config = LoraConfig(
    r = LORA_R,
    bias = LORA_BIAS,
    lora_alpha = LORA_ALPHA,
    task_type = LORA_TASK_TYPE,
    lora_dropout = LORA_DROPOUT,
    target_modules = LORA_TARGET_MODULES
)

### Training Configuration

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    save_steps = 0,
    fp16 = TRAINER_FP16,
    bf16 = TRAINER_BF16,
    optim = TRAINER_OPTIM,
    load_best_model_at_end = True,
    max_steps = TRAINER_MAX_STEPS,
    eval_steps = TRAINER_EVAL_STEPS,
    output_dir = TRAINER_OUTPUT_DIR,
    metric_for_best_model = 'eval_loss',
    weight_decay = TRAINER_WEIGHT_DECAY,
    warmup_ratio = TRAINER_WARMUP_RATIO,
    logging_steps = TRAINER_LOGGING_STEPS,
    learning_rate = TRAINER_LEARNING_RATE,
    max_grad_norm = TRAINER_MAX_GRAD_NORM,
    group_by_length = TRAINER_GROUP_BY_LENGTH,
    lr_scheduler_type = TRAINER_LR_SCHEDULER_TYPE,
    evaluation_strategy = TRAINER_EVALUATION_STRATEGY,
    per_device_train_batch_size = TRAINER_PER_DEVICE_TRAIN_BATCH_SIZE,
    gradient_accumulation_steps = TRAINER_GRADIENT_ACCUMULATION_STEPS,

)

In [ ]:
FINETUNED_MODEL = "Hawk28/CodeLlama-13B-RP-2"

### SFT Trainer

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model = model,
    packing = SFT_PACKING,
    tokenizer = tokenizer,
    peft_config = peft_config,
    args = training_arguments,
    dataset_text_field = "prompt",
    # dataset_text_field = "prompts",
    train_dataset = dataset["train"],
    eval_dataset = dataset['validation'],
    max_seq_length = SFT_MAX_SEQ_LENGTH
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/8659 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

### Model Training

In [ ]:
# Train model
trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a CodeLlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,0.725600,0.674106
50,0.668800,0.604153
75,0.429500,0.575726
100,0.569400,0.601344
125,0.375300,0.570772
150,0.452300,0.613655
175,0.306700,0.594877
200,0.358400,0.639660
225,0.255600,0.609445
250,0.338200,0.642007


TrainOutput(global_step=300, training_loss=0.4168199650446574, metrics={'train_runtime': 2069.6143, 'train_samples_per_second': 1.16, 'train_steps_per_second': 0.145, 'total_flos': 3.819860425285632e+16, 'train_loss': 0.4168199650446574, 'epoch': 0.28})

In [ ]:
# Save trained model
trainer.model.save_pretrained(FINETUNED_MODEL)

### Inference

In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# continents ( ContId, Continent )
# countries ( CountryId, CountryName, Continent )
# car_makers ( Id, Maker, FullName, Country )
# model_list ( ModelId, Maker, Model )
# car_names ( MakeId, Model, Make )
# cars_data ( Id, MPG, Cylinders, Edispl, Horsepower, Weight, Accelerate, Year )
#
# countries.Continent can be joined with continents.ContId
# car_makers.Country can be joined with countries.CountryId
# model_list.Maker can be joined with car_makers.Id
# car_names.Model can be joined with model_list.Model
# cars_data.Id can be joined with car_names.MakeId
#
### Question:
#
# For each continent, list its id, name, and how many countries it has?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# continents ( ContId, Continent )
# countries ( CountryId, CountryName, Continent )
# car_makers ( Id, Maker, FullName, Country )
# model_list ( ModelId, Maker, Model )
# car_names ( MakeId, Model, Make )
# cars_data ( Id, MPG, Cylinders, Edispl, Horsepower, Weight, Accelerate, Year )
#
# countries.Continent can be joined with continents.ContId
# car_makers.Country can be joined with countries.CountryId
# model_list.Maker can be joined with car_makers.Id
# car_names.Model can be joined with model_list.Model
# cars_data.Id can be joined with car_names.MakeId
#
### Question:
#
# For each continent, list its id, name, and how many countries it has?
#
### SQL:
#
# SELECT ContId ,  Continent ,  COUNT(*) FROM countries GROUP BY ContId ,  Continent
#
### End.
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#



In [ ]:
# batch = tokenizer("""Below is an instruction that describes a task, paired with an input that provides further context. Write an SQL query that appropriately completes the request. Make sure to use the table relationships when joining two or more tables.

# ### Instruction:

# For each continent, list its id, name, and how many countries it has?

# ### Input:

# CREATE TABLE continents (ContId NUMBER PRIMARY KEY, Continent TEXT); CREATE TABLE countries (CountryId NUMBER PRIMARY KEY, CountryName TEXT, Continent NUMBER); CREATE TABLE car_makers (Id NUMBER PRIMARY KEY, Maker TEXT, FullName TEXT, Country TEXT); CREATE TABLE model_list (ModelId NUMBER PRIMARY KEY, Maker NUMBER, Model TEXT); CREATE TABLE car_names (MakeId NUMBER PRIMARY KEY, Model TEXT, Make TEXT); CREATE TABLE cars_data (Id NUMBER PRIMARY KEY, MPG TEXT, Cylinders NUMBER, Edispl NUMBER, Horsepower TEXT, Weight NUMBER, Accelerate NUMBER, Year NUMBER);

# ### Table Relationships:

# countries Continent RELATES TO continents ContId; car_makers Country RELATES TO countries CountryId; model_list Maker RELATES TO car_makers Id; car_names Model RELATES TO model_list Model; cars_data Id RELATES TO car_names MakeId

# ### Response:
# """, return_tensors='pt')

# with torch.cuda.amp.autocast():
#     input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
#     output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

# print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

## Inferences

In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and genres of all the books?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    attention_mask = torch.ones(input_ids.shape, dtype=torch.bool).to("cuda")
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100, attention_mask=attention_mask)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and genres of all the books?
#
### SQL:
#
# SELECT title ,  genre FROM books
#
### End.
#
# 
# 
# 
# 
# 
### Question:
#
# What are the names of all the authors?
#
### SQL:
#
# SELECT author_name FROM authors
#
### End.
# 
# 
# 
# 
# 
### Question:
# 



In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the number of books published in each genre
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the number of books published in each genre
#
### SQL:
#
# SELECT COUNT(*) ,  genre FROM books GROUP BY genre
#
### End.
#
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
#


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the most borrowed book
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the most borrowed book
#
### SQL:
#
# SELECT book_id FROM borrowedbooks GROUP BY book_id ORDER BY count(*) DESC LIMIT 1
#
### End.
#
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 



In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the IDs, titles, and publication years of "Fantasy" genre books?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the IDs, titles, and publication years of "Fantasy" genre books?
#
### SQL:
#
# SELECT book_id ,  title ,  publication_year FROM books WHERE genre  =  "Fantasy"
#
### End.
#
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 



In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of book titles and their respective authors?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of book titles and their respective authors?
#
### SQL:
#
# SELECT title ,  author_name FROM books AS T1 JOIN authors AS T2 ON T1.author_id  =  T2.author_id
#
### End.
#
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 



In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Which members borrowed books along with the borrow and return dates?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Which members borrowed books along with the borrow and return dates?
#
### SQL:
#
# SELECT T1.member_name ,  T2.borrow_date ,  T2.return_date FROM members AS T1 JOIN borrowedbooks AS T2 ON T1.member_id  =  T2.member_id
#
### End.
#
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and authors of books by authors born after 1950?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and authors of books by authors born after 1950?
#
### SQL:
#
# SELECT T1.title ,  T2.author_name FROM books AS T1 JOIN authors AS T2 ON T1.author_id  =  T2.author_id WHERE T2.birth_year  >  1950
#
### End.
#
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 



In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Could you provide a list of unique genres available in the collection?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Could you provide a list of unique genres available in the collection?
#
### SQL:
#
# SELECT DISTINCT genre FROM books
#
### End.
#
# 
# 
# 
# 
# 
### Question:
#
# How many books are there in the collection?
#
### SQL:
#
# SELECT count(*) FROM books
#
### End.
# 
# 
# 
# 
# 
### Question:
#
# What


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Show the members who joined before 1st January 2022 (Format: DD-MM-YYYY) and the books they borrowed:
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Show the members who joined before 1st January 2022 (Format: DD-MM-YYYY) and the books they borrowed:
#
### SQL:
#
# SELECT T1.member_name ,  T2.title FROM members AS T1 JOIN borrowedbooks AS T3 ON T1.member_id  =  T3.member_id JOIN books AS T2 ON T3.book_id  =  T2.book_id WHERE T1.join_date  <  "2022-01-01"
#
### End.
#
# 



In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of members along with the number of books they have borrowed, sorted by the number of borrows in descending order?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of members along with the number of books they have borrowed, sorted by the number of borrows in descending order?
#
### SQL:
#
# SELECT member_name ,  COUNT(*) FROM borrowedbooks GROUP BY member_id ORDER BY COUNT(*) DESC
#
### End.
#
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What is the earliest publication year of a book in the "Fiction" genre?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What is the earliest publication year of a book in the "Fiction" genre?
#
### SQL:
#
# SELECT MIN(publication_year) FROM books WHERE genre  =  "Fiction"
#
### End.
#
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
#


### Save Model on HuggingFace

In [ ]:
# Empty VRAM
del model
del trainer
del batch
import gc
gc.collect()
gc.collect()

0

In [ ]:
BASE_MODEL_NAME

'codellama/CodeLlama-13b-hf'

In [ ]:
FINETUNED_MODEL

'Hawk28/CodeLlama-13B-RP-2'

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    low_cpu_mem_usage = True,
    return_dict = True,
    torch_dtype = torch.float16,
    device_map = DEVICE_MAP,
    trust_remote_code = True
)
model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model.push_to_hub(FINETUNED_MODEL)
tokenizer.push_to_hub(FINETUNED_MODEL)

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Hawk28/CodeLlama-13B-RP-2/commit/61a99f30947153a758c3ea078686d31c184fe5b2', commit_message='Upload tokenizer', commit_description='', oid='61a99f30947153a758c3ea078686d31c184fe5b2', pr_url=None, pr_revision=None, pr_num=None)